In [44]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import time
import os
import pickle
import joblib
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import getpass
serviceKey = getpass.getpass("Enter Open API Key: ")

Enter Open API Key: ··········


In [ ]:
url_area = 'http://apis.data.go.kr/B553457/nopenapi/rest/publicperformancedisplays/area'
url_subject = 'http://apis.data.go.kr/B553457/nopenapi/rest/publicperformancedisplays/realm'
url_period = 'http://apis.data.go.kr/B553457/nopenapi/rest/publicperformancedisplays/period'
url_detail = 'http://apis.data.go.kr/B553457/nopenapi/rest/publicperformancedisplays/detail'

In [ ]:
# API 호출결과를 dataFrame으로 만들어주는 함수
def make_df(res):
    content = res.text
    obj = BeautifulSoup(content, 'lxml-xml')
    rows = obj.findAll('item')
    row_list = []
    name_list = []
    value_list = []

    for i in range(0, len(rows)):
        columns = rows[i].find_all()
        for j in range(0,len(columns)):
            if i ==0:
                name_list.append(columns[j].name)
            value_list.append(columns[j].text)
        row_list.append(value_list)
        value_list=[]

    data = pd.DataFrame(row_list, columns=name_list)
    return data

In [ ]:
def get_info(row, page):

  params ={'serviceKey' : serviceKey,
          'numOfrows': row,
          'PageNo' : page }

  res_area = requests.get(url_area, params=params, verify=False)
  infor_df = make_df(res_area)

  return infor_df


In [21]:
def get_detail(seq):

  params ={'serviceKey' : serviceKey,
          'seq': seq
  }
  res_detail = requests.get(url_detail, params=params, verify=False)

  return make_df(res_detail)

In [77]:
get_info(100,31) # numofrows, page / 마지막 데이터 확인

,serviceName,seq,title,startDate,endDate,place,realmName,area,sigungu,thumbnail,gpsX,gpsY
0,교육/체험,190035,[2.1.~2.4. 사전접수]박물관 이리 온(on)_삼한시대 구슬 디자인으로 마스크...,,,,교육/체험,,,http://www.culture.go.kr/upload/rdf//upload/ho...,,
1,교육/체험,190036,"[미션블루 웹드라마_5] 선박, 아는만큼 안전하다",,,,교육/체험,,,http://www.culture.go.kr/upload/rdf//upload/ho...,,
2,교육/체험,190037,낙동강에서 만난 물고기 친구들! - 낙동강 어류채집,,,,교육/체험,,,http://www.culture.go.kr/upload/rdf//upload/ho...,,
3,교육/체험,190055,내 맘대로 색깔 꽃,,,,교육/체험,,,http://www.culture.go.kr/upload/rdf//upload/ho...,,


In [81]:
# 전체 데이터 조회
df = pd.DataFrame()
for i in range(1, 32):
    data = get_info(100, i)
    if i % 10 == 0:
      print(len(df))
    time.sleep(1)
    df = pd.concat([df, data])

df = df.reset_index(drop = True)
df = df.iloc[:, 1:]

900
1900
2900


In [82]:
len(df)

3004

In [83]:
# 상세 정보 조회
detail_dataframes = pd.DataFrame()
for index, row in df.iterrows():
    detail_df = get_detail(row['seq'])
    detail_dataframes = pd.concat([detail_dataframes, detail_df])
    time.sleep(0.1)
    if index % 300 == 0:
      print(f"{index}번째 행 처리 완료")
    #if index - 10 == 0:
    #  break

0번째 행 처리 완료
300번째 행 처리 완료
600번째 행 처리 완료
900번째 행 처리 완료
1200번째 행 처리 완료
1500번째 행 처리 완료
1800번째 행 처리 완료
2100번째 행 처리 완료
2400번째 행 처리 완료
2700번째 행 처리 완료
3000번째 행 처리 완료


In [84]:
merge = pd.merge(df, detail_dataframes, on = 'seq', how = 'left')

merge_selected = merge[['seq', 'title_x', 'startDate_x', 'endDate_x', 'place_x', 'realmName_x',
       'area_x', 'sigungu_x', 'thumbnail', 'gpsX_x', 'gpsY_x', 'price', 'contents1', 'placeAddr']]

merge_selected.columns = ['seq', 'title', 'startDate', 'endDate', 'place', 'realmName',
       'area', 'sigungu', 'thumbnail', 'gpsX', 'gpsY', 'price', 'contents1', 'placeAddr']

print(len(merge_selected))

3004


In [85]:
merge_selected

,seq,title,startDate,endDate,place,realmName,area,sigungu,thumbnail,gpsX,gpsY,price,contents1,placeAddr
0,250002,토비아스 레베르거: 가끔이나마 내 마음대로 할 수 있는 곳은 나 자신 뿐이다,20180616,20251231,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/23/07/show...,128.9427422591895,35.10921642590141,무료,,부산광역시 사하구 낙동남로 1191 부산 현대미술관
1,319005,패트릭 블랑: 수직정원,20180616,20281231,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/25/04/show...,128.9427422591895,35.10921642590141,무료,,부산광역시 사하구 낙동남로 1191 부산 현대미술관
2,267899,"홍영철의 유산, 부산영화를 담다",20181005,20301231,부산영화체험박물관,전시,부산,중구,http://www.culture.go.kr/upload/rdf/24/01/show...,129.03371136372274,35.10174011607882,"성인 10,000원 / 어린이, 청소년 7,000원",,부산광역시 중구 대청로126번길 12 부산영화체험박물관
3,247148,[국립중앙도서관 상설 전시] 시간의 기록을 잇다,20181029,20251231,국립중앙도서관,전시,서울,서초구,http://www.culture.go.kr/upload/rdf/23/06/show...,127.00298547116853,37.49761794050167,무료,,서울특별시 서초구 반포대로 201 국립중앙도서관
4,300600,밀양아리랑 전시관 상설전시,20190314,20251231,밀양아리랑아트센터,전시,경남,밀양시,http://www.culture.go.kr/upload/rdf/24/11/show...,128.7584103654324,35.50441176799155,무료,,경상남도 밀양시 밀양대공원로 112 밀양아리랑아트센터
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,189404,서초문화재단 화요 콘서트 「하이든 교향곡 전곡 시리즈Ⅰ」,,,반포심산아트홀,음악/콘서트,서울,서초구,http://www.culture.go.kr/upload/rdf//upload/ho...,126.990515657787,37.4988531475655,,<p>아시아 최초 하이든 전곡 연주 프로젝트 &quot;하이든 107곡 전곡 프로젝...,서울특별시 서초구 사평대로 55 심산기념문화센터
3000,190035,[2.1.~2.4. 사전접수]박물관 이리 온(on)_삼한시대 구슬 디자인으로 마스크...,,,,교육/체험,,,http://www.culture.go.kr/upload/rdf//upload/ho...,,,,"<p style=""text-align:center""><strong><span sty...",
3001,190036,"[미션블루 웹드라마_5] 선박, 아는만큼 안전하다",,,,교육/체험,,,http://www.culture.go.kr/upload/rdf//upload/ho...,,,,<p>선박에서 반드시 지켜야 할 해양안전수칙을 웹드라마 형식으로 제작하였습니다.</...,
3002,190037,낙동강에서 만난 물고기 친구들! - 낙동강 어류채집,,,,교육/체험,,,http://www.culture.go.kr/upload/rdf//upload/ho...,,,,<p>낙동강에서 족대를 이용한 어류채집을 다녀왔습니다!</p>\r\n\r\n<p>날...,


In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
import pandas as pd

# Define the filename and path in your Google Drive
filename = 'merge_selected_data.csv'
save_path = f'/content/drive/MyDrive/빅데이터자연어처리프로젝트/data/{filename}'

# Save the DataFrame to CSV
merge_selected.to_csv(save_path, index=False)

print(f"'{filename}' has been saved to your Google Drive at '{save_path}'")

'merge_selected_data.csv' has been saved to your Google Drive at '/content/drive/MyDrive/빅데이터자연어처리프로젝트/data/merge_selected_data.csv'


In [88]:
pd.read_csv(save_path)

,seq,title,startDate,endDate,place,realmName,area,sigungu,thumbnail,gpsX,gpsY,price,contents1,placeAddr
0,250002,토비아스 레베르거: 가끔이나마 내 마음대로 할 수 있는 곳은 나 자신 뿐이다,20180616.0,20251231.0,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/23/07/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관
1,319005,패트릭 블랑: 수직정원,20180616.0,20281231.0,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/25/04/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관
2,267899,"홍영철의 유산, 부산영화를 담다",20181005.0,20301231.0,부산영화체험박물관,전시,부산,중구,http://www.culture.go.kr/upload/rdf/24/01/show...,129.033711,35.101740,"성인 10,000원 / 어린이, 청소년 7,000원",NaN,부산광역시 중구 대청로126번길 12 부산영화체험박물관
3,247148,[국립중앙도서관 상설 전시] 시간의 기록을 잇다,20181029.0,20251231.0,국립중앙도서관,전시,서울,서초구,http://www.culture.go.kr/upload/rdf/23/06/show...,127.002985,37.497618,무료,NaN,서울특별시 서초구 반포대로 201 국립중앙도서관
4,300600,밀양아리랑 전시관 상설전시,20190314.0,20251231.0,밀양아리랑아트센터,전시,경남,밀양시,http://www.culture.go.kr/upload/rdf/24/11/show...,128.758410,35.504412,무료,NaN,경상남도 밀양시 밀양대공원로 112 밀양아리랑아트센터
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,189404,서초문화재단 화요 콘서트 「하이든 교향곡 전곡 시리즈Ⅰ」,NaN,NaN,반포심산아트홀,음악/콘서트,서울,서초구,http://www.culture.go.kr/upload/rdf//upload/ho...,126.990516,37.498853,NaN,<p>아시아 최초 하이든 전곡 연주 프로젝트 &quot;하이든 107곡 전곡 프로젝...,서울특별시 서초구 사평대로 55 심산기념문화센터
3000,190035,[2.1.~2.4. 사전접수]박물관 이리 온(on)_삼한시대 구슬 디자인으로 마스크...,NaN,NaN,NaN,교육/체험,NaN,NaN,http://www.culture.go.kr/upload/rdf//upload/ho...,NaN,NaN,NaN,"<p style=""text-align:center""><strong><span sty...",NaN
3001,190036,"[미션블루 웹드라마_5] 선박, 아는만큼 안전하다",NaN,NaN,NaN,교육/체험,NaN,NaN,http://www.culture.go.kr/upload/rdf//upload/ho...,NaN,NaN,NaN,<p>선박에서 반드시 지켜야 할 해양안전수칙을 웹드라마 형식으로 제작하였습니다.</...,NaN
3002,190037,낙동강에서 만난 물고기 친구들! - 낙동강 어류채집,NaN,NaN,NaN,교육/체험,NaN,NaN,http://www.culture.go.kr/upload/rdf//upload/ho...,NaN,NaN,NaN,<p>낙동강에서 족대를 이용한 어류채집을 다녀왔습니다!</p>\r\n\r\n<p>날...,NaN
